# Práctica 1 - Map-Reduce - RDD
---

#### 1. De una lista de 20 números enteros obtenga la suma de los pares

In [1]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/16 19:42:18 WARN Utils: Your hostname, MacBook-Pro-de-Pablo.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.84 instead (on interface en0)
25/08/16 19:42:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/16 19:42:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd = spark.sparkContext.parallelize(range(1, 21))

In [5]:
def pares(elemento):
    if elemento % 2 == 0:
        return elemento
    return 0

In [6]:
def sumar_pares(x, y):
    return x + y

In [7]:
rdd.map(pares).reduce(sumar_pares)

110

#### 2. En un laboratorio se realizó una encuesta. A cada miembro del laboratorio se le hizo la siguiente pregunta: ¿A quién, de todos sus compañeros de trabajo, considera que es su amigo? El resultado de la encuesta se almacenó en el archivo “friends.txt”. Utilizando map-reduce debe contar la cantidad de amigos que tiene cada miembro del laboratorio.

| Salida esperada |
| --              |
| Jim,3           |
| Joe,1           |
| Kai,1           |
| Lin,2           |
| Sue,1           |

In [8]:
friends = spark.sparkContext.textFile('friends.txt')

In [9]:
friends.collect()

['Jim,Sue',
 'Sue,Jim',
 'Lin,Joe',
 'Joe,Lin',
 'Jim,Kai',
 'Kai,Jim',
 'Jim,Lin',
 'Lin,Jim']

In [10]:
def limpiar(elemento):
    return elemento.strip().split(",")

In [11]:
def agregar_claves(elemento):
    return (elemento[1], 1)

In [12]:
def reducir(x, y):
    return x + y

In [13]:
mapeado = friends.map(limpiar)
mapeado.collect()

[['Jim', 'Sue'],
 ['Sue', 'Jim'],
 ['Lin', 'Joe'],
 ['Joe', 'Lin'],
 ['Jim', 'Kai'],
 ['Kai', 'Jim'],
 ['Jim', 'Lin'],
 ['Lin', 'Jim']]

In [14]:
mapeado = mapeado.map(agregar_claves)
mapeado.collect()

[('Sue', 1),
 ('Jim', 1),
 ('Joe', 1),
 ('Lin', 1),
 ('Kai', 1),
 ('Jim', 1),
 ('Lin', 1),
 ('Jim', 1)]

In [15]:
final = mapeado.reduceByKey(reducir)
final.collect()

[('Jim', 3), ('Sue', 1), ('Joe', 1), ('Lin', 2), ('Kai', 1)]

#### 3. El archivo "kaonashi14M.txt" contiene contraseñas de uso muy frecuente (algunas no tanto). Se desea obtener la longitud de cada contraseña. Se debe clasificar cada contraseña de acuerdo a la cantidad de caracteres teniendo como referencia lo siguiente:
* tiny: 1 small: 2–4 medium: 5–9 big: de 10 en adelante
* Con map-reduce debe clasificar cada contraseña y luego contar la cantidad de contraseñas que hay en cada categoría

| Salida esperada |
| --              |
| tiny,1          |
| small,70        |
| medium,120      |
| big,50          |

In [25]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [26]:
contrasenias = spark.sparkContext.textFile('kaonashi14M.txt')

In [27]:
contrasenias.collect()

['123456',
 '123456789',
 'qwerty',
 'password',
 '12345',
 'x',
 'qwerty123',
 '1q2w3e',
 'DEFAULT',
 '12345678',
 '111111',
 '1234567890',
 'victory',
 '1983',
 'y',
 'orlando',
 'dpbk1234',
 'aspirine',
 'agent007',
 '7894561230',
 'rocky1',
 'assassin',
 '12413',
 '987456321',
 '12345678901',
 '000',
 'ncc1701',
 '9999999',
 'sammy',
 'z',
 'spiderman1',
 'abc1234',
 'PE#5GZ29PTZMSE',
 'sunflower',
 'teacher',
 '19911991',
 '<password>',
 'potter',
 'microsoft',
 '567890',
 'cowboys1',
 'steve',
 'online',
 'asdfgh1',
 'pierre',
 'qwertz',
 'xx',
 'tintin',
 '3rjs1la7qe',
 'julian',
 'jasmin',
 '4321',
 '123456987',
 'vfhecz',
 'newmember',
 'disney',
 'christina',
 'dthjybrf',
 'k123456',
 '5xfgs3Ii9D',
 'Passw0rd',
 'claire',
 'admin123',
 '1981',
 'chocolat',
 'packers',
 '20102010',
 'thebest',
 'mexico1',
 '19901990',
 'startrek',
 'lover1',
 'mother1',
 'sayang',
 'francesca',
 'diesel',
 'newyork1',
 'domino',
 'anderson',
 'anhyeuem',
 'sophia',
 'indian',
 'angel123',
 'ra

In [28]:
def clasificar(elemento):
    n = len(elemento)
    if n == 1:
        return ('tiny', 1)
    if n >= 2 and n <= 4:
        return ('small', 1)
    if n >= 5 and n <= 9:
        return ('medium', 1)
    return ('big', 1)

In [29]:
def reducir(x, y):
    return x + y

In [30]:
mapeado = contrasenias.map(clasificar)
mapeado.collect()

[('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('tiny', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('small', 1),
 ('tiny', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('big', 1),
 ('small', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('tiny', 1),
 ('big', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('small', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('medium', 1),
 ('small', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('big', 1),
 ('medium', 1),
 ('medium', 1),
 ('medium', 1),
 ('small', 1),
 ('medium',

In [31]:
final = mapeado.reduceByKey(reducir)
final.collect()

[('tiny', 3), ('big', 10), ('small', 5), ('medium', 86)]